<h1 style = "font-size: 30px; text-align: center;">AI Genetic Hands On</h1>
<h2 style = "font-size: 25px; text-align: center;">Hospital Job Scheduling</h2>
<h2 style = "font-size: 25px; text-align: center; color: #666">Name: Daneshvar Amrollahi</h2>
<h2 style = "font-size: 25px; text-align: center; color: #666">Student Id: 810197685</h2>
<h4 style="text-align: center">Spring 1400</h4>

<h2 style = "font-size: 25px;">Notes</h2>

<h3>Tests: </h3>

<p style="text-indent :2em;"><b>Test1: </b><mark>Average time < 15s</mark></p>
<p style="text-indent :2em;"><b>Test2: </b><mark>Average time < 60s</mark></p>
    

<h3>Outputs: </h3>
<p style="text-indent :2em;">Respectively <mark>output1.txt</mark> and <mark>output2.txt</mark></p>

<br>

<h2 style = "font-size: 25px;">Test Files</h2>

In [1]:
testFile1 = "test1.txt"
testFile2 = "test2.txt"

<h2 style = "font-size: 25px;">Reading Test File Content</h2>

In [115]:
def readInput(testFile) :
    file = open(testFile, 'r+')
    fileList = file.readlines()
    fileList = [s.replace('\n', '') for s in fileList]
    
    [days, doctors] = [int(i) for i in fileList[0].split()]
    maxCapacity = int(fileList[1])
    
    allShifts = []
    for i in range(2, days + 2):
        dayRequirements = fileList[i].split()
        morningReqs = [int(i) for i in dayRequirements[0].split(",")]
        eveningReqs = [int(i) for i in dayRequirements[1].split(",")]
        nightReqs = [int(i) for i in dayRequirements[2].split(",")]
        
        allShifts.append((morningReqs, eveningReqs, nightReqs))

    file.close()
    return [days, list(range(doctors)), maxCapacity, allShifts]

<h2 style = "font-size: 25px;">Job Scheduler</h2>

In [165]:
from random import choice
import random
from math import floor

class JobScheduler:
    def __init__(self, fileInfo):
        self.days = fileInfo[0]
        self.doctors = len(fileInfo[1])
        self.doctorsIds = fileInfo[1]
        self.maxCapacity = fileInfo[2]
        self.allShifts = fileInfo[3]
        
        

        # You can use these values for your genetic algorithm
        self.popSize = 300
        self.crossOverPoints = (self.doctors // 5)
        self.elitismPercentage = 16 #(move x% best of parents directly to the new population)
        self.pc = 0.65 #(crossover probability)
        self.pm = 0.05  #(mutation probability)
        
        self.chromosomes = self.generateInitialPopulation()
        
        self.goalReached = 0
        self.ans = []
        
    def generateInitialPopulation(self):
        initPop = []
        for t in range(self.popSize):
            chromosome = []
            for i in range(self.doctors):
                row = []
                for j in range(self.days):
                    row.append(choice(['m', 'e', 'n', 'o'])) #morning, evening, night, off
                chromosome.append(row)
            
                
            #for i in range(self.doctors):
            #    for j in range(self.days):
            #        print(chromosome[i][j], end = " ")
            #    print()
            
            #print()
            
            initPop.append(chromosome)
        
        
        return initPop
        
    
    def crossOver(self, chromosome1, chromosome2):
            
        col = [0] * self.days #col[i] = X: pick that column from parX
        for t in range(self.crossOverPoints):
            colNum = random.randint(0, self.days - 1)
            prob = random.random()
            if (prob < self.pc):
                col[colNum] = 1 - col[colNum]

        child = []
        for i in range(self.doctors):
            row = []
            for j in range(self.days):
                if (col[j] == 0):
                    row.append(chromosome1[i][j])
                else:
                    row.append(chromosome2[i][j])

            child.append(row)
            
        return child
        
                
    def mutate(self, chromosome):
        for i in range(self.doctors):
            for j in range(self.days):
                prob = random.random()
                if (prob < self.pm):
                    chromosome[i][j] = choice(['m', 'e', 'n', 'o'])
        
        return chromosome
        
        
    def calculateCost(self, chromosome):
        c1 = 0 #number of violations on capacities for each doctor

        
        for i in range(self.doctors):
            morning = chromosome[i].count('m')
            evening = chromosome[i].count('e')
            night = chromosome[i].count('n')
            if (morning + evening + night > self.maxCapacity):
                c1 += 1
        
        c2 = 0 #min-max violations on each shift
        for j in range(self.days):
            morning = 0
            evening = 0
            night = 0
            for i in range(self.doctors):
                morning += (chromosome[i][j] == 'm')
                evening += (chromosome[i][j] == 'e')
                night += (chromosome[i][j] == 'n')
            
            if (not (self.allShifts[j][0][0] <= morning and morning <= self.allShifts[j][0][1]) ):
                c2 += 1
                
            if (not (self.allShifts[j][1][0] <= evening and evening <= self.allShifts[j][1][1]) ):
                c2 += 1
                
            if (not (self.allShifts[j][2][0] <= night and night <= self.allShifts[j][2][1]) ):
                c2 += 1
        
        c3 = 0 #violations on shifts
        for i in range(self.doctors):
            for j in range(self.days):
                if (j >= 1):
                    c3 += (chromosome[i][j] == 'm' and chromosome[i][j - 1] == 'n')
                    c3 += (chromosome[i][j] == 'e' and chromosome[i][j - 1] == 'n')
                if (j >= 2):
                    c3 += (chromosome[i][j] == 'n' and chromosome[i][j - 1] == 'n' and chromosome[i][j - 2] == 'n')
        
        
        
        w1, w2, w3 = 1, 1, 1
        
        return w1 * c1 + w2 * c2 + w3 * c3
    
    
    def generateNewPopulation(self):
        curPop = self.chromosomes
        
        curChromosomes = []
        for chromosome in self.chromosomes:
            #print("current chromosome:")
            #print(chromosome)
            curChromosomes.append((self.calculateCost(chromosome), chromosome))
        
        curChromosomes.sort() #based on cost
        
        cost = curChromosomes[0][0]
        #print(cost)
        
        goodChromosomes = curChromosomes[:floor((self.elitismPercentage / 100) * (self.popSize))]
        if (curChromosomes[0][0] == 0): #no violations
            self.goalReached = 1
            self.ans = curChromosomes[0][1]
            return
        
        
        
        newPop = [chromosome[1] for chromosome in goodChromosomes]
        
        while (len(newPop) < self.popSize):
            par0, par1 = random.sample(goodChromosomes, 2)
            child = self.crossOver(par0[1], par1[1])
            mutatedChild = self.mutate(child)
            
            newPop.append(mutatedChild)
            
        return newPop
    
    
    def schedule(self): 
        while (not(self.goalReached)):
            newPop = self.generateNewPopulation()
            self.chromosomes = newPop
        print(self.goalReached)
        return self.ans

In [158]:
a = [5, 1, 5, 20, 5]
a

[5, 1, 5, 20, 5]

<h2 style = "font-size: 25px;">Execution</h2>

In [166]:
import time

fileInfo1 = readInput(testFile1)

print(fileInfo1[0])
print(fileInfo1[1])
print(fileInfo1[2])
print(fileInfo1[3])

start = time.time()

scheduler = JobScheduler(fileInfo1)
ans = scheduler.schedule()

print(ans)

for i in range(len(ans)):
    for j in range(len(ans[0])):
        print(ans[i][j], end = " ")
    print()
print()

end = time.time()

print("test 1: ", '%.2f'%(end - start), 'sec')

10
[0, 1, 2, 3, 4, 5, 6]
8
[([2, 5], [1, 2], [2, 3]), ([1, 4], [2, 2], [1, 3]), ([1, 5], [1, 5], [1, 1]), ([1, 1], [1, 2], [3, 3]), ([1, 2], [0, 1], [1, 1]), ([1, 1], [0, 1], [1, 3]), ([2, 2], [1, 1], [1, 1]), ([0, 1], [1, 2], [2, 2]), ([0, 1], [0, 2], [0, 3]), ([1, 2], [1, 1], [2, 3])]
1
[['e', 'e', 'e', 'n', 'o', 'o', 'e', 'e', 'o', 'n'], ['m', 'o', 'e', 'n', 'o', 'o', 'o', 'o', 'o', 'm'], ['n', 'o', 'm', 'o', 'o', 'o', 'm', 'o', 'n', 'n'], ['m', 'e', 'n', 'o', 'm', 'n', 'o', 'n', 'o', 'n'], ['m', 'o', 'o', 'm', 'o', 'e', 'm', 'm', 'o', 'o'], ['n', 'n', 'o', 'n', 'n', 'o', 'n', 'o', 'e', 'e'], ['m', 'm', 'm', 'e', 'm', 'm', 'o', 'n', 'n', 'o']]
e e e n o o e e o n 
m o e n o o o o o m 
n o m o o o m o n n 
m e n o m n o n o n 
m o o m o e m m o o 
n n o n n o n o e e 
m m m e m m o n n o 

test 1:  0.47 sec


In [167]:
fileInfo2 = readInput(testFile2)

start = time.time()

scheduler = JobScheduler(fileInfo2)
scheduler.schedule()

end = time.time()

print("Test 2: ", '%.2f'%(end - start), 'sec')

1
Test 2:  5.27 sec
